In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import cv2
import matplotlib.pyplot as plt


import hydra
from typing import Optional
from collections.abc import Callable
from omegaconf import OmegaConf, DictConfig
from data_module.lightning_data_module import DataModule
from hydra import initialize, initialize_config_module, initialize_config_dir, compose

from model_module.lightining_model_module import ModelModule

In [2]:
def setup(cfg):

    cfg.loader.batch_size = 1

    if 'split' not in cfg:
        cfg.split = 'val'

    if 'shuffle' not in cfg:
        cfg.shuffle = False
        
def setup_config(cfg: DictConfig, override: Optional[Callable] = None):

    OmegaConf.set_struct(cfg, False)
    OmegaConf.resolve(cfg)
    OmegaConf.set_struct(cfg, True)

In [3]:
# *  config setup  * #
CONFIG_PATH = '/ws/CV_For_Autonomous_Driving/config'

with initialize_config_dir(config_dir=CONFIG_PATH):
    cfg = compose(config_name='default_config_debug.yaml')
    
setup_config(cfg, setup)

# dataset list 만드는 test
DM = DataModule('nuscenes_generated', cfg.data, cfg.loader)
val_dataloader = DM.val_dataloader()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  after removing the cwd from sys.path.


In [4]:
MM = ModelModule()
MM.cuda()


2023-01-12 08:51:02,006 - mmcv - INFO - initialize SECONDFPN with init_cfg [{'type': 'Kaiming', 'layer': 'ConvTranspose2d'}, {'type': 'Constant', 'layer': 'NaiveSyncBatchNorm2d', 'val': 1.0}]
2023-01-12 08:51:02,016 - mmcv - INFO - 
deblocks.0.0.weight - torch.Size([128, 256, 4, 4]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-12 08:51:02,017 - mmcv - INFO - 
deblocks.0.1.weight - torch.Size([128]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-12 08:51:02,018 - mmcv - INFO - 
deblocks.0.1.bias - torch.Size([128]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-12 08:51:02,018 - mmcv - INFO - 
deblocks.1.0.weight - torch.Size([128, 512, 2, 2]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2023-01-12 08:51:02,019 - mmcv - INFO - 
deblocks.1.1.weight - torch.Size([128]): 
The value is the same before and after calling `init_weig

ModelModule(
  (fullmodel): BaseBEVDepth(
    (backbone): BaseLSSFPN(
      (img_backbone): ResNet(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): ResLayer(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, af

In [5]:
data = next(iter(val_dataloader))

In [6]:
det_loss, depth_loss = MM.shared_step(data)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch

In [6]:
pred, depth = MM(data)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [17]:
'''
model에 클래스 별로 head가 있음
클래스는 lightning_model_module.py에 있음

샘플이 별로 없는 클래스들은 하나로 묶어서 준다..

pred: 각 head 별 6개의 dict
    reg torch.Size([8, 2, 128, 128])        # offset..
    height torch.Size([8, 1, 128, 128])     # BEV height
    dim torch.Size([8, 3, 128, 128])        # 3D bbox w, h, l
    rot torch.Size([8, 2, 128, 128])        # sin(alpha), cos(alpha)
    vel torch.Size([8, 2, 128, 128])        # vx, vy
    heatmap torch.Size([8, 1, 128, 128])    # center confidence
'''

print(len(pred))
for cam in range(6):
    for k, v in pred[cam][0].items():
        print(k, v.shape)
    print(pred[cam][0].keys())
# print(pred[0].shape)
print(depth.shape)

6
reg torch.Size([8, 2, 128, 128])
height torch.Size([8, 1, 128, 128])
dim torch.Size([8, 3, 128, 128])
rot torch.Size([8, 2, 128, 128])
vel torch.Size([8, 2, 128, 128])
heatmap torch.Size([8, 1, 128, 128])
dict_keys(['reg', 'height', 'dim', 'rot', 'vel', 'heatmap'])
reg torch.Size([8, 2, 128, 128])
height torch.Size([8, 1, 128, 128])
dim torch.Size([8, 3, 128, 128])
rot torch.Size([8, 2, 128, 128])
vel torch.Size([8, 2, 128, 128])
heatmap torch.Size([8, 2, 128, 128])
dict_keys(['reg', 'height', 'dim', 'rot', 'vel', 'heatmap'])
reg torch.Size([8, 2, 128, 128])
height torch.Size([8, 1, 128, 128])
dim torch.Size([8, 3, 128, 128])
rot torch.Size([8, 2, 128, 128])
vel torch.Size([8, 2, 128, 128])
heatmap torch.Size([8, 2, 128, 128])
dict_keys(['reg', 'height', 'dim', 'rot', 'vel', 'heatmap'])
reg torch.Size([8, 2, 128, 128])
height torch.Size([8, 1, 128, 128])
dim torch.Size([8, 3, 128, 128])
rot torch.Size([8, 2, 128, 128])
vel torch.Size([8, 2, 128, 128])
heatmap torch.Size([8, 1, 128, 1